In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Conversando com os dados

In [17]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
caminhos = [
    'arquivos/Explorando a API da OpenAI.pdf'
]

paginas = []
for caminho in caminhos:
    loader = PyPDFLoader(caminho)
    paginas.extend(loader.load())

recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10,
    separators=['\n\n', '\n', '.', ' ', '']
)

documents = recur_split.split_documents(paginas)

In [20]:
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=OpenAIEmbeddings()
)

## Criando estrutura de conversa

In [ ]:
from langchain_core.prompts import ChatPromptTemplate as cpt

prompt = cpt.from_template(
    '''Responda as perguntas se baseando no contexto fornecido
    
    contexto: {contexto}
    
    pergunta: {pergunta}''')

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':5, 'fetch_k': 25})
setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
})

In [ ]:
input = setup.invoke('O que é a OpenAI?')
input

{'pergunta': 'O que é a OpenAI?',
 'contexto': [Document(metadata={'source': 'arquivos/Explorando a API da OpenAI.pdf', 'page': 9}, page_content='Explorando a API da OpenAI\nGuias\nTambém está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da\nOpenAI.\nAsimov Academy 9'),
  Document(metadata={'source': 'arquivos/Explorando a API da OpenAI.pdf', 'page': 23}, page_content="ChatCompletion da biblioteca da OpenAI:\nimport openai\nfrom dotenv import load_dotenv, find_dotenv\n_ = load_dotenv(find_dotenv())\nclient = openai.Client()\nmensagens = [{ 'role': 'user', 'content': 'O que é uma maçã em até 5 palavras? '}]\nresposta = client.chat.completions.create(\nmessages=mensagens,\nmodel='gpt-3.5-turbo-0125',\nmax_tokens=1000,\ntemperature=0,\n)\nprint(type(resposta))\nEla possui algumas propriedades interessantes:\nAsimov Academy 23"),
  Document(metadata={'source': 'arquivos/Explorando a API da OpenAI.pdf', 'page': 6}, page_content='Explorando a API da OpenAI\n

In [24]:
def join_documents(input):
    input['contexto'] = '\n\n'.join([c.page_content for c in input['contexto']])
    return input

setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
}) | join_documents

input = setup.invoke('O que é a OpenAI?')
input

{'pergunta': 'O que é a OpenAI?',
 'contexto': "Explorando a API da OpenAI\nGuias\nTambém está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da\nOpenAI.\nAsimov Academy 9\n\nChatCompletion da biblioteca da OpenAI:\nimport openai\nfrom dotenv import load_dotenv, find_dotenv\n_ = load_dotenv(find_dotenv())\nclient = openai.Client()\nmensagens = [{ 'role': 'user', 'content': 'O que é uma maçã em até 5 palavras? '}]\nresposta = client.chat.completions.create(\nmessages=mensagens,\nmodel='gpt-3.5-turbo-0125',\nmax_tokens=1000,\ntemperature=0,\n)\nprint(type(resposta))\nEla possui algumas propriedades interessantes:\nAsimov Academy 23\n\nExplorando a API da OpenAI\nA OpenAI\nNosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência\nartificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela\nempresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:

In [25]:
print(input['contexto'])

Explorando a API da OpenAI
Guias
Também está disponível diversos guias de boas práticas recomendadas ao utilizar os modelos da
OpenAI.
Asimov Academy 9

ChatCompletion da biblioteca da OpenAI:
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = openai.Client()
mensagens = [{ 'role': 'user', 'content': 'O que é uma maçã em até 5 palavras? '}]
resposta = client.chat.completions.create(
messages=mensagens,
model='gpt-3.5-turbo-0125',
max_tokens=1000,
temperature=0,
)
print(type(resposta))
Ela possui algumas propriedades interessantes:
Asimov Academy 23

Explorando a API da OpenAI
A OpenAI
Nosso objetivo é explorar a API da OpenAI, a maior desenvolvedora de ferramentas de inteligência
artificial do mundo na atualidade. O modelo ChatGPT é apenas um dos vários modelos criados pela
empresa nos últimos anos. A seguir, apresentamos alguns dos principais modelos e suas funções:
• GPT-4: Trata-se de um modelo de linguagem de grande escala (LLM) multim

In [26]:
from langchain_openai.chat_models import ChatOpenAI

chat = ChatOpenAI(model='gpt-4o-mini')

chain = setup | prompt | chat
resposta = chain.invoke('O que é a OpenAI?')
resposta.content

'A OpenAI é a maior desenvolvedora de ferramentas de inteligência artificial do mundo atualmente. Ela é conhecida por criar diversos modelos de IA, incluindo o ChatGPT, que é um modelo de linguagem de grande escala. A empresa tem como objetivo explorar e desenvolver tecnologias de inteligência artificial, oferecendo diversas ferramentas e APIs que permitem aos desenvolvedores criar aplicações inovadoras. Além do ChatGPT, a OpenAI também desenvolve modelos como o GPT-4, DALL-E, TTS e Whisper, cada um com suas próprias funcionalidades e aplicações.'

## Juntando tudo

In [ ]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate as cpt
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI

load_dotenv()

prompt = cpt.from_template(
    '''Responda as perguntas se baseando no contexto fornecido
    
    contexto: {contexto}
    
    pergunta: {pergunta}''')

retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={'k':5, 'fetch_k': 25})

def join_documents(input):
    input['contexto'] = '\n\n'.join([c.page_content for c in input['contexto']])
    return input

setup = RunnableParallel({
    'pergunta': RunnablePassthrough(),
    'contexto': retriever
}) | join_documents

chat = ChatOpenAI(model='gpt-4o-mini')

chain = setup | prompt | chat
resposta = chain.invoke('O que é a OpenAI?')
resposta.content